In [2]:
# currently having trouble importing urbana so i'm just copying the src-code

import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.feature_selection import (
    RFE,
    SelectKBest,
    mutual_info_regression,
    f_regression,
    SelectPercentile,
)
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import (
    train_test_split,
    cross_validate,
    RepeatedKFold,
    GridSearchCV,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer, StandardScaler


from urbana.features import normaltests
from urbana.constants import DIR_REPO, DIR_DATA, RANDOM_STATE
from urbana.models.plot_predictions import PredictedAccuracy


def LinearModel(YEAR, MONTH, VARIABLE_TO_PREDICT):

    mpl.use('Agg')

    print("Starting with {}-{:02d}".format(YEAR, MONTH))


    OUTPUT_WARNINGS = False
    SAVE_FIGS = True
    SAVE_MODEL = True

    K_EDUCATION = 1
    K_AGE = 2
    K_NATIONALITY = 2
    K_RENT = 1
    K_POI = 10


    if not OUTPUT_WARNINGS:
        import warnings

        warnings.filterwarnings("ignore")


    ALLOWED_YEARS = [2017, 2018, 2023]

    if YEAR not in ALLOWED_YEARS:
        raise Exception("Please select a year within: {}".format(ALLOWED_YEARS))

    if YEAR == 2018 and MONTH == 3:
        raise Exception(
            "Month 3 (March) is not available for 2018. Please choose a different month."
        )


    # Create folders to store the data

    DIR_VAR = DIR_DATA / "processed/{}".format(VARIABLE_TO_PREDICT)
    DIR_MONTH = DIR_VAR / "{}_{:02d}".format(YEAR, MONTH)
    DIR_LINEAR = DIR_MONTH / "01_linear"

    if SAVE_FIGS or SAVE_MODEL:
        folder_list = [DIR_VAR, DIR_MONTH, DIR_LINEAR, DIR_VAR / "01_linear"]

        import os

        for folder in folder_list:
            if not os.path.exists(folder):
                os.makedirs(folder)


    PATH_TO_FILE = DIR_DATA / "interim/sections_{}_{:02d}.csv".format(YEAR, MONTH)
    if os.path.isfile(PATH_TO_FILE) is False:
        raise Exception(
            'Please run first the notebook "00acquisition.ipynb" with the same date and "SAVE_DATA" set to True'
        )

    np.random.seed(RANDOM_STATE)

    sect = pd.read_csv(
        DIR_DATA / "interim/sections_{}_{:02d}.csv".format(YEAR, MONTH),
    )

    sect.set_index("Tag", inplace=True)

    sect.drop(["N_district", "N_neighbourhood", "N_section"], axis=1, inplace=True)



    y = sect[VARIABLE_TO_PREDICT]

    X = sect.drop(
        ["Airbnb_Number", "Airbnb_Price", "Airbnb_Price_Person", "Airbnb_Location_Score"],
        axis=1,
    )



    geo_info = gpd.read_file(DIR_DATA / "interim/sections_geo.json")

    geo_info.set_index("Tag", inplace=True)

    geo_info[VARIABLE_TO_PREDICT] = sect[VARIABLE_TO_PREDICT]

    #print("Area with maximum value: " + str(geo_info[VARIABLE_TO_PREDICT].idxmax()))



    fig, ax = plt.subplots(figsize=(20, 20))

    geo_info.plot(
        ax=ax,
        column=VARIABLE_TO_PREDICT,
        legend=True,
        figsize=(20, 20),
        legend_kwds={"shrink": 0.7},
    )

    ax.set_title("Target variable: " + str(VARIABLE_TO_PREDICT), fontsize=20, y=1.01)

    if SAVE_FIGS:
        plt.savefig(DIR_LINEAR / "target_variable.svg", format="svg")


    # Check which variables are already normal
    normality_test = normaltests.get_normaltest_df(X.T)

    pt = PowerTransformer()
    preprocessor = Pipeline(steps=[("imputer", KNNImputer()), ("pt", pt)])

    X_Education = X.filter(regex="^Education")

    kbest_Education = SelectKBest(f_regression, k=K_EDUCATION).fit(
        preprocessor.fit_transform(X_Education),
        pt.fit_transform(y.values.reshape(-1, 1)),
    )

    education_cols = kbest_Education.get_support(indices=True)
    X_Education_chosen = X_Education.columns[education_cols]
    #X_Education_chosen



    X_Age = X.filter(regex="^Percentage_Age_")
    print(X_Age)
    kbest_Age = SelectKBest(f_regression, k=K_AGE).fit(
        preprocessor.fit_transform(X_Age),
        pt.fit_transform(y.values.reshape(-1, 1)),
    )

    age_cols = kbest_Age.get_support(indices=True)
    X_Age_chosen = X_Age.columns[age_cols]
    #X_Age_chosen



    X_Nationality = X.filter(regex="^Nationality_")
    X_Nationality.drop(["Nationality_Spain"], axis=1, inplace=True)
    kbest_Nationality = SelectKBest(f_regression, k=K_NATIONALITY).fit(
        preprocessor.fit_transform(X_Nationality),
        pt.fit_transform(y.values.reshape(-1, 1)),
    )

    nationality_cols = kbest_Nationality.get_support(indices=True)
    X_Nationality_chosen = X_Nationality.columns[nationality_cols]
    #X_Nationality_chosen

    if YEAR >= 2015 and YEAR <= 2018:

        X_Rent = X.filter(regex="^Rent_")
        kbest_Rent = SelectKBest(f_regression, k=K_RENT).fit(
            preprocessor.fit_transform(X_Rent),
            pt.fit_transform(y.values.reshape(-1, 1)),
        )

        rent_cols = kbest_Rent.get_support(indices=True)
        X_Rent_chosen = X_Rent.columns[rent_cols]
        #X_Rent_chosen




    X_POI = X.filter(regex="^POI")

    kbest_POI = SelectKBest(f_regression, k=K_POI).fit(
        preprocessor.fit_transform(X_POI),
        pt.fit_transform(y.values.reshape(-1, 1)),
    )

    POI_cols = kbest_POI.get_support(indices=True)
    X_POI_chosen = X_POI.columns[POI_cols]
    #X_POI_chosen


    if YEAR >= 2015 and YEAR <= 2018:
        X.drop(np.setdiff1d(X_Rent.columns, X_Rent_chosen), axis=1, inplace=True)

    X.drop(np.setdiff1d(X_Age.columns, X_Age_chosen), axis=1, inplace=True)

    X.drop(np.setdiff1d(X_Nationality.columns, X_Nationality_chosen), axis=1, inplace=True)

    X.drop(np.setdiff1d(X_Education.columns, X_Education_chosen), axis=1, inplace=True)

    X.drop(np.setdiff1d(X_POI.columns, X_POI_chosen), axis=1, inplace=True)


    # ## Feature Selection Pipeline

    # In order to perform a feature selection, we will use *RFE* (Recursive Feature Elimination).
    # 
    # The number of variables to use will be a hyper-paramater that will be tuned with a GridSearch using RMSE as the metric.
    # 
    # The target feature will also be transformed with a PowerTransfomrer, by applying the *TransformedTargetRegressor*.


    # Define the regressor to use
    myRegressor = LinearRegression()

    # Define a pipeline with the preprocessing, feature selection (RFE) and regressor
    pipe_rfe = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("rfe", RFE(estimator=myRegressor)),
            ("regressor", myRegressor),
        ]
    )

    # Define the param space for hyper-parameter tunning (in this case, the number of features to keep with RFE)
    param_grid_rfe = [{"rfe__n_features_to_select": np.arange(6, 15, 1)}]

    search_rfe = GridSearchCV(
        pipe_rfe, param_grid_rfe, scoring="neg_root_mean_squared_error", n_jobs=-1
    )


    model = TransformedTargetRegressor(regressor=search_rfe, transformer=PowerTransformer())

    model.fit(X, y)



    #print("Best Model:")
    #print(
    #    "Number of features: "
    #    + str(model.regressor_.best_params_["rfe__n_features_to_select"])
    #)
    #print("\nList of features:")
    cols_rfe = model.regressor_.best_estimator_.named_steps["rfe"].get_support(indices=True)
    #print(X.columns[cols_rfe])



    score_features = -model.regressor_.cv_results_["mean_test_score"]
    n_features = []
    for i in model.regressor_.cv_results_["params"]:
        n_features.append(i["rfe__n_features_to_select"])

    id_min_score = score_features.argmin()

    fig, ax = plt.subplots(figsize=(15, 10))
    plt.plot(n_features, score_features, marker="o")
    plt.axvline(x=n_features[id_min_score], color=".5")

    ax.set_xlabel("Number of features", fontsize=15)
    ax.set_ylabel("Median Absolute Error", fontsize=15)
    ax.set_xticks(np.arange(min(n_features), max(n_features) + 1))
    ax.set_title("Score by number of features", fontsize=20, y=1.01)

    if SAVE_FIGS:
        plt.savefig(DIR_LINEAR / "selection_rmse.svg", format="svg")

    plt.close()



    y_pred_rfe = model.predict(X).round()
    pa_rfe = PredictedAccuracy(y, y_pred_rfe)
    pa_rfe.plot_scatter(save_fig=SAVE_FIGS, root_name=DIR_LINEAR / "model"), y=1.01)


    if SAVE_FIGS:
        plt.savefig(DIR_LINEAR / "relative_errors.svg", format="svg")

    plt.close()

    if SAVE_MODEL:
        geo_info[["Chosen_Error"]].to_csv(DIR_LINEAR / "relative_errors.csv")
        df_predictions = pd.DataFrame(y_pred_rfe, index=geo_info.index, columns=["Predictions"])
        df_predictions.to_csv(DIR_LINEAR / "predictions.csv")  
        df_predictions.to_csv(DIR_VAR / "01_linear/predictions_{}_{:02d}.csv".format(YEAR, MONTH))  

    # from yellowbrick.regressor.residuals import residuals_plot

    # residuals_plot(
    #     model, X_train, y_train, X_test, y_test, hist=False, qqplot=True, is_fitted=True
    # )

    # residuals_plot(
    #     model, X_train, y_train, X_test, y_test, hist=True, qqplot=False, is_fitted=True
    # )


    pw = PowerTransformer()
    pw.fit(y.values.reshape(-1, 1))

    ####################Tranform y_hat####################
    y_pred_transformed = model.predict(X)
    y_pred_transformed = pw.transform(y_pred_transformed.reshape(-1, 1)).flatten()

    ####################Trasform y####################
    # y_test_transformed = pd.Series(pw.transform(y_test.values.reshape(-1, 1)).flatten())
    y_transformed = pd.Series(pw.transform(y.values.reshape(-1, 1)).flatten())
    y_transformed.name = "Transformed Airbnb_Number"

    pa_trans = PredictedAccuracy(y_transformed, y_pred_transformed)
    pa_trans.plot_scatter(

    ####################Tranform y_hat####################
    y_pred_transformed = model.predict(X)
    y_pred_transformed = pw.transform(y_pred_transformed.reshape(-1, 1)).flatten()

    ####################Trasform y####################
    # y_test_transformed = pd.Series(pw.transform(y_test.values.reshape(-1, 1)).flatten())
    y_transformed = pd.Series(pw.transform(y.values.reshape(-1, 1)).flatten())
    y_transformed.name = "Transformed Airbnb_Number"

    pa_trans = PredictedAccuracy(y_transformed, y_pred_transformed)
    pa_trans.plot_scatter(
        save_fig=SAVE_FIGS,
        root_name=DIR_LINEAR / "transformed_model",
    )
    pa_trans.plot_errors(
        save_fig=SAVE_FIGS,
        root_name=DIR_LINEAR / "transformed_model",
    )
    del pa_trans


    # # Sensitivity Analysis


    X_rfe = X.iloc[:, cols_rfe]

    pipe_sens = Pipeline(steps=[("preprocessor", preprocessor), ("regressor", myRegressor)])

    model_sens = TransformedTargetRegressor(
        regressor=pipe_sens, transformer=PowerTransformer()
    )

    model_sens.fit(X_rfe, y)



    cv_rfe = cross_validate(
        model_sens,
        X_rfe,
        y,
        cv=RepeatedKFold(n_splits=5, n_repeats=5),
        scoring=["neg_root_mean_squared_error"],
        return_estimator=True,
        n_jobs=-1,
    )

    coefs_rfe = pd.DataFrame(
        [est.regressor_.named_steps["regressor"].coef_ for est in cv_rfe["estimator"]],
        columns=X_rfe.columns,
    )


    coefs_rfe["Intercept"] = pd.Series(
        [est.regressor_.named_steps["regressor"].intercept_ for est in cv_rfe["estimator"]]
    )

    medians_rfe = coefs_rfe.drop(["Intercept"], axis=1).median()
    medians_rfe = medians_rfe.reindex(medians_rfe.abs().sort_values(ascending=False).index)
    medians_rfe = pd.concat(
        [medians_rfe, pd.Series({"Intercept": None}, index=["Intercept"])]
    )
    coefs_rfe = coefs_rfe[medians_rfe.index]

    limit_value = (
        max(abs(coefs_rfe.to_numpy().min()), abs(coefs_rfe.to_numpy().max())) * 1.05
    )

    print(coefs_rfe)

    fig, ax = plt.subplots(figsize=(20, 20))

    sns.stripplot(ax=ax, data=coefs_rfe, orient="h", color="k", alpha=0.5)
    sns.boxplot(ax=ax, data=coefs_rfe, orient="h", color="cyan", saturation=0.5)
    plt.axvline(x=0, color="red")

    plt.figtext(0.51, 0.9, "Linear Model: Coefficient robustness", fontsize=20, ha="center")
    plt.figtext(
        0.51,
        0.885,
        "{}-{:02d}".format(YEAR, MONTH),
        fontsize=18,
        ha="center",
    )
    ax.set_xlim(-limit_value, limit_value)
    ax.set_xlabel("Coefficient value", fontsize=15)

    if SAVE_FIGS:
        plt.savefig(DIR_LINEAR / "sensitivity.svg", format="svg")
        plt.savefig(DIR_VAR / "01_linear/sensitivity_{}_{:02d}.svg".format(YEAR, MONTH), format="svg")
        

    plt.close()




    if SAVE_MODEL:
        coefs_rfe.to_csv(DIR_LINEAR / "coefficients.csv")
        coefs_rfe.to_csv(DIR_VAR / "01_linear/coefficients_{}_{:02d}.csv".format(YEAR, MONTH))

    print("Done with {}-{:02d}".format(YEAR, MONTH))
    print("##################################")


In [4]:
LinearModel(2023, 12, "POI_Education")

Starting with 2023-12
line 223
        Percentage_Age_0_14  Percentage_Age_15_24  Percentage_Age_25_39  \
Tag                                                                       
01_001             0.123172              0.087760              0.326405   
01_002             0.101376              0.126720              0.356264   
01_003             0.200466              0.130637              0.268548   
01_004             0.122317              0.111414              0.336627   
01_005             0.118053              0.112452              0.316674   
...                     ...                   ...                   ...   
10_143             0.122778              0.110556              0.201111   
10_234             0.091419              0.087493              0.199103   
10_235             0.105837              0.066148              0.167315   
10_236             0.091536              0.067712              0.183699   
10_237             0.111720              0.101862              0.1938

In [3]:
YEAR, MONTH = 2023, 12
path_to_file = DIR_DATA / "interim/sections_{}_{:02d}.csv".format(YEAR, MONTH)
df = pd.read_csv(path_to_file)
all_features = df.columns
# skipping section coding and nationality features
independent_features = all_features[5:38]
nan_columns = df[independent_features].apply(lambda col: col.isna().values.any())
features_to_use = independent_features[~nan_columns]

for feature in features_to_use[:1]:
    print(f'running with model with {feature}')
    LinearModel(YEAR, MONTH, feature)
    

running with model with POI_Daily_Food
Starting with 2023-12
line 223
        Percentage_Age_0_14  Percentage_Age_15_24  Percentage_Age_25_39  \
Tag                                                                       
01_001             0.123172              0.087760              0.326405   
01_002             0.101376              0.126720              0.356264   
01_003             0.200466              0.130637              0.268548   
01_004             0.122317              0.111414              0.336627   
01_005             0.118053              0.112452              0.316674   
...                     ...                   ...                   ...   
10_143             0.122778              0.110556              0.201111   
10_234             0.091419              0.087493              0.199103   
10_235             0.105837              0.066148              0.167315   
10_236             0.091536              0.067712              0.183699   
10_237             0.111720   

ver relaciones interesantes:
- airbnb nunber: distance-center, poi restaurants/hotels, nationality UK, edad 25-39
- edades0-14: distance-center, edad25-39, education-none, household?
- edades25-39: espagnoles, age65+, distance-center
- souvenirs_stores: distance-center, rent-price
- 